In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999

##### I will be using different source for scraping instead of Wiki. See URL below.

In [2]:
tables = pd.read_html('https://www.zipcodesonline.com/2020/06/postal-code-of-toronto-in-2020.html')

In [3]:
tn_df = tables[1]
tn_df.head()

,0,1,2,3
0,SL. NO.,NEIGHBOURHOOD,POSTAL CODES,DISTRICT
1,NaN,NaN,NaN,NaN
2,1,Adelaide,M5H,Downtown Toronto
3,2,Agincourt North,M1V,Scarborough
4,3,Agincourt,M1S,Scarborough


##### Droping  non useful rows 0 and 1.

In [4]:
tn_df2 = tn_df.drop(index =[0,1]).drop(columns=0)

##### Changing columns names.

In [5]:
tn_df3 = tn_df2.reset_index(drop=True)
tn_df3.columns = ['Neighborhood','PostalCode','Borough']

In [6]:
tn_df4 = tn_df3[['PostalCode','Borough','Neighborhood']]
tn_df4

,PostalCode,Borough,Neighborhood
0,M5H,Downtown Toronto,Adelaide
1,M1V,Scarborough,Agincourt North
2,M1S,Scarborough,Agincourt
3,M9V,Etobicoke,Albion Gardens
4,M8W,Etobicoke,Alderwood
5,M3H,North York,Bathurst Manor
6,M5V,Downtown Toronto,Bathurst Quay
7,M2K,North York,Bayview Village
8,M9V,Etobicoke,Beaumond Heights
9,M5M,North York,"Bedford Park, ,"


##### Now we need to clean data in column PostalCodes. Three rows have 2+ postal codes togeather. So, we will brake them down. *We will write indexes of rows which we are braking down to be sute to drop them later easily.

In [7]:
drop_indx = []
multy_pc_df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
for indx, series in tn_df4.iterrows():
    if len(series.PostalCode) > 3:
        drop_indx.append(series.name)
        postal_codes = series.PostalCode.split(',')
        for pc in postal_codes:
            multy_pc_df = multy_pc_df.append({'PostalCode' : pc,
                                                'Borough' : series.Borough,
                                                'Neighborhood' : series.Neighborhood},
                                               ignore_index=True)
print(drop_indx)
multy_pc_df

[40, 77, 197]


,PostalCode,Borough,Neighborhood
0,M3M,North York,Downsview
1,M3L,North York,Downsview
2,M3N,North York,Downsview
3,M9B,Etobicoke,Islington Avenue
4,M9A,Etobicoke,Islington Avenue
5,M2M,North York,Willowdale
6,M2N,North York,Willowdale
7,M2R,North York,Willowdale


##### Now concat new broken DF with main DF where we droping those 3 rows which we have broken down.

In [8]:
tn_df5 = pd.concat([multy_pc_df, tn_df4.drop(drop_indx)])
tn_df5 = tn_df5.reset_index(drop=True)

##### Calculating unique PostalCodes before grouping by Postal Code and Borough to be sure we haven't missed any of them after groupping and function for combining neighborhoods with the same postal code in one row. 

In [9]:
tn_df5.nunique()

PostalCode      103
Borough          11
Neighborhood    201
dtype: int64

##### Combining function. Puts all neighborhoods with the same postal code to the same row. Then we check if all rows has been preserved. We have 103 - the same as qunique values from the main frame.

In [10]:
def combine_n(group):
    (group_label, df) = group
    if df.shape[1] > 1:
        neighborhoods = df.Neighborhood.tolist()
        df.iloc[0, df.columns.get_loc('Neighborhood')] = ', '.join(neighborhoods)
        df2 = df.iloc[0]
        return df2
    else:
        return df.iloc[0]

groups = tn_df5.groupby(by=['PostalCode','Borough'])
data = [combine_n(g) for g in groups]

combined_df = pd.DataFrame(data)
combined_df.shape   

(103, 3)

##### Last touches: Reseting index and text aligning.

In [11]:
comb_toronto_df = combined_df.reset_index(drop=True)
comb_toronto_df.style.set_properties(**{'text-align': 'left'})

,PostalCode,Borough,Neighborhood
0,M2R,North York,Willowdale
1,M9A,Etobicoke,Islington Avenue
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn
6,M1H,Scarborough,Cedarbrae
7,M1J,Scarborough,Scarborough Village
8,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
9,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


In [12]:
comb_toronto_df.to_pickle('toronto-neighborhoods-df.pkl')